In [ ]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
costarican = pd.read_csv(r"../input/costa-rican-household-poverty-prediction/train.csv")

In [ ]:
costaricanTest = pd.read_csv(r"../input/costa-rican-household-poverty-prediction/test.csv")

In [ ]:
costarican.shape

In [ ]:
costarican.head()

In [ ]:
costaricanTest.head()

In [ ]:
print(costarican.isnull().sum())

In [ ]:
XCR = costarican.drop(columns=['Id','v2a1','v18q1','rez_esc','idhogar','Target'])
XCR.head()

In [ ]:
XCRtest = costaricanTest.drop(columns=['Id','v2a1','v18q1','rez_esc','idhogar'])
XCRtest.head()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [ ]:
XCR.shape

In [ ]:
YCR = costarican.Target

In [ ]:
XCR.edjefe = XCR.edjefe.eq('yes').mul(1)
XCR.dependency = XCR.dependency.eq('yes').mul(1)
XCR.edjefa = XCR.edjefa.eq('yes').mul(1)

In [ ]:
XCRtest.edjefe = XCRtest.edjefe.eq('yes').mul(1)
XCRtest.dependency = XCRtest.dependency.eq('yes').mul(1)
XCRtest.edjefa = XCRtest.edjefa.eq('yes').mul(1)

In [ ]:
XCR.edjefe = XCR.edjefe.eq('no').mul(0)
XCR.dependency = XCR.dependency.eq('no').mul(0)
XCR.edjefa = XCR.edjefa.eq('no').mul(0)

In [ ]:
XCRtest.edjefe = XCRtest.edjefe.eq('no').mul(0)
XCRtest.dependency = XCRtest.dependency.eq('no').mul(0)
XCRtest.edjefa = XCRtest.edjefa.eq('no').mul(0)

In [ ]:
for a in XCR.columns:
    XCR[a].fillna(XCR[a].mode()[0],inplace=True)

In [ ]:
for b in XCRtest.columns:
    XCRtest[b].fillna(XCRtest[b].mode()[0],inplace=True)

In [ ]:
best = 0
media_anterior = 0
for i in range(1,50):
    knn = KNeighborsClassifier(n_neighbors = i)
    scores = cross_val_score(knn, XCR, YCR, cv=10)
    media = sum(scores)/len(scores)
    if media > media_anterior:
        media_anterior = media
        best = i
print(media_anterior)
print(best)

In [ ]:
knn = KNeighborsClassifier(n_neighbors = best)

In [ ]:
knn.fit(XCR,YCR)

In [ ]:
YtestPred = knn.predict(XCRtest)

In [ ]:
Pred = pd.DataFrame(YtestPred)
Pred.columns = ['Target']
Pred.insert(0, 'Id', costaricanTest.Id , True)
Pred.head()

In [ ]:
Pred.to_csv('CRPrediction.csv', header=True, index=False)